In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from config1 import weather_api_key 
import random
import requests

In [2]:
#Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
coordinates = zip(lats,lngs)
coordinates_lst = list(coordinates)
coordinates_lst



[(-34.31923565328853, 176.39331486770743),
 (-30.699028264055215, -53.89789771492393),
 (-20.60530390558715, -127.9794695283395),
 (-6.5925260782015584, -122.26077145686314),
 (-27.587676550144963, -33.90026523914608),
 (-60.286063287971004, -23.592983869398438),
 (89.91817114957547, 123.15471122730179),
 (56.24897081271067, 118.60081932087309),
 (-56.977275024141726, -17.46351652221884),
 (-37.05121059808715, -19.721556788558786),
 (25.401387977022154, 7.693519604373961),
 (-62.38301718325665, 174.4444032896197),
 (-28.76189118885131, 32.43654105179584),
 (40.14733789944685, 106.82012230938136),
 (63.22971374948753, 125.53812198632966),
 (-78.96577899835641, -47.50616997675766),
 (49.55519024579377, -60.98601940690925),
 (-15.150518347184843, 162.93448181941653),
 (27.211731958791788, 141.26413905399966),
 (45.20481321883264, 88.14052541422365),
 (12.617045922353398, -172.4843638419585),
 (-46.15995375349526, -125.33003891272752),
 (-70.19279564885508, 126.0904763147438),
 (-26.536919

In [3]:
 cities = []
for coordinate in coordinates_lst:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)

len(cities)

790

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+ weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=3807f646ceb305ae314ef537d740df20


In [5]:
cities_data = []

for i, city in enumerate(cities):
    try:
        city_url = url + "&q=" + city.replace(" ","+")
        city_data_json = requests.get(city_url).json()
        city_lat = city_data_json["coord"]["lat"]
        city_lon = city_data_json["coord"]["lon"]
        city_max_temp = city_data_json["main"]["temp_max"]
        city_per_hum = city_data_json["main"]["humidity"]
        city_per_cloud = city_data_json["clouds"]["all"]
        city_wind = city_data_json["wind"]["speed"]
        city_country = city_data_json["sys"]["country"]
        city_desc = city_data_json["weather"]
        for elem in city_desc:      #accedemos a cada elemento de la lista (en este caso cada elemento es un dictionario)
            for k,v in elem.items():
                if k == "description":
                    city_desc = v 
                
         
        cities_data.append({"City": city.title(),
                            "Country":city_country,
                            "Lat": city_lat,
                            "Long": city_lon,
                            "Max temp": city_max_temp,
                            "Humidity": city_per_hum,
                            "Cloudness": city_per_cloud,
                            "Wind S": city_wind,
                           "Current Description":city_desc})
        #print(i)
    except:
        pass
        
            

In [6]:
cities_data_df = pd.DataFrame(cities_data)

In [7]:
cities_data_df.head()

,City,Country,Lat,Long,Max temp,Humidity,Cloudness,Wind S,Current Description
0,Ngunguru,NZ,-35.6167,174.5000,58.15,78,100,2.80,light rain
1,Cacapava Do Sul,BR,-30.5122,-53.4914,82.74,36,0,4.32,clear sky
2,Rikitea,PF,-23.1203,-134.9692,72.81,78,100,3.80,overcast clouds
3,Atuona,PF,-9.8000,-139.0333,77.43,68,30,14.50,scattered clouds
4,Sao Joao Da Barra,BR,-21.6403,-41.0511,84.29,79,73,13.29,broken clouds


In [8]:
output_data_file = "cities.csv"
cities_data_df.to_csv(output_data_file, index_label="City_ID")